In [1]:
import monai.transforms as mt
import logging
import sys
import matplotlib.pyplot as plt
import ignite
import numpy as np
import torch
import monai
import warnings
warnings.filterwarnings("ignore")  # remove some scikit-image warnings

In [2]:
def mDice(image,Ground_truth_mask,predictor,prompt = "box"):
    Dice_lst = []
    local_gt = Ground_truth_mask.astype("uint8")
    if np.max(local_gt) == 0:
        return 1
    for i in range(1,np.max(local_gt)+1):
        i_gt_mask = np.where(local_gt == i, 1, 0)
        if np.sum(i_gt_mask.flatten()) == 0:
            continue
        else:
            predictor.set_image(image)
            if prompt == "point":
                # 随机选一个点
                indices = np.nonzero(i_gt_mask)
                random_index = np.random.randint(0,len(indices[0])-1)
                point = np.array([(indices[1][random_index],indices[0][random_index])])
                SAM_mask , scores, logits = predictor.predict(point_coords=point,point_labels=np.array([1]),multimask_output=False,)
            elif prompt == "points":
                # 随机选5个点
                indices = np.nonzero(i_gt_mask)
                random_index = np.random.random_integers(0,len(indices[0])-1,5)
                points = []
                for j in range(len(random_index)):
                    points.append((indices[1][random_index[j]],indices[0][random_index[j]]))
                points = np.array(points)
                SAM_mask , scores, logits = predictor.predict(point_coords=points,point_labels=np.ones(len(points)),multimask_output=False,)
            elif prompt == "box":
                indices = np.nonzero(i_gt_mask)
                x_min = min(indices[1])
                x_max = max(indices[1])
                y_min = min(indices[0])
                y_max = max(indices[0])
                input_box = np.array([x_min,y_min,x_max,y_max])
                SAM_mask , scores, logits = predictor.predict(point_coords=None,point_labels=None,box=input_box,multimask_output=False,)
            else:
                raise NameError("prompt should be in [\"point\",\"points\",\"box\"]")
            Dice_lst.append(Dice(SAM_mask,i_gt_mask))        
    return sum(Dice_lst) / len(Dice_lst)
  

In [3]:
from monai.apps import datasets
import json
import os
from monai import transforms
import torchvision
from monai.transforms import LoadImaged,EnsureChannelFirstd,Compose,ToTensord
import logging

image_transform = Compose([
    LoadImaged(keys=("image", "label")),
    torchvision.transforms.Resize(( 224 , 224 )),
    torchvision.transforms.Grayscale(num_output_channels=3),
    ToTensord(keys=("image", "label"))
])

with open("./data/dataset_0.json") as file1:
    dataset = json.load(file1)

#train_dataset =  monai.data.CacheDataset(dataset["training"], transform=image_transform)
train_dataset =  monai.data.Dataset(dataset["training"], transform=image_transform)
train_loader  = monai.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, drop_last=True)

In [ ]:
from segment_anything import SamPredictor, sam_model_registry 
import torch 
import os 

# 根据检查点加载模型
sam_model = sam_model_registry["vit_h"](checkpoint="./checkpoint/sam_vit_h_4b8939.pth")
sam_model.to(device=device)
predictor = SamPredictor(sam_model)

# 定义损失函数和优化器
optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters()) 
criterion = torch.nn.BCEWithLogitsLoss()





In [ ]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader,0):
        print("load successfully!")
        inputs = data.to(device)
        
        optimizer.zero_grad()

        outputs = predictor.sam(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
PATH = "finetune/fine_tuned_sam.pth"
torch.save(model.state_dict(), PATH)